In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests

In [167]:
holdings=['QQQ','NVO','AZN','MBUU','CWH']

In [54]:
for ticker in holdings:
    d={'function': 'STOCH','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'interval':'daily','fastkperiod':22 }
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )
    slowk=[]
    slowd=[]
    for key, val in response.json()['Technical Analysis: STOCH'].items():
        slowk.append(float(val['SlowK']))
        slowd.append(float(val['SlowD']))
        
    sto=pd.DataFrame({'Date':list(response.json()['Technical Analysis: STOCH'].keys()),
                 'k':slowk,
                  'd':slowd
                 })
    sto=sto.astype({'Date': 'datetime64'})
    sto.sort_values(by='Date',inplace=True)
    sto['k_change'] = sto.k/sto.k.shift(3)-1
    print(sto)

           Date        k        d  k_change
5006 2000-06-22  91.0542  94.2856       NaN
5005 2000-06-23  83.5821  90.2647       NaN
5004 2000-06-26  76.6470  83.7611       NaN
5003 2000-06-27  72.5592  77.5961 -0.203121
5002 2000-06-28  72.8650  74.0237 -0.128222
...         ...      ...      ...       ...
4    2020-05-12  91.8402  95.3003 -0.029197
3    2020-05-13  80.0734  89.8020 -0.170811
2    2020-05-14  72.7483  81.5540 -0.253805
1    2020-05-15  72.9614  75.2610 -0.205561
0    2020-05-18  83.2266  76.3121  0.039379

[5007 rows x 4 columns]


In [168]:
for ticker in holdings:
    
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )
    spy=[[],[],[],[],[],[],[]]
    for key, val in response.json()['Time Series (Daily)'].items():
        spy[0].append(float(val['1. open']))
        spy[1].append(float(val['2. high']))
        spy[2].append(float(val['3. low']))
        spy[3].append(float(val['4. close']))
        spy[4].append(float(val['5. adjusted close']))
        #spy[5].append(float(val['6. volume']))
        #spy[6].append(float(val['7. dividend amount']))
        
    spy=pd.DataFrame({'Date':list(response.json()['Time Series (Daily)'].keys()),
                 'open':spy[0],
                  'high':spy[1],
                  'low':spy[2],
                  'close':spy[3],
                  'adj_close':spy[4]#,
                #  'volume':spy[5],
                #  'dividend':spy[6],
                 })
    spy=spy.astype({'Date': 'datetime64'})
    spy=spy.sort_values(by='Date')
    spy['highest_high']=spy.high.rolling(22).max()
    spy['lowest_low']=spy.low.rolling(22).min()
    spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
    spy['k']=spy.k.rolling(3).mean()
    spy['k_change'] = spy.k/spy.k.shift(3)-1
    print(ticker,spy.iloc[-1]['Date'], spy.iloc[-1]['k_change'],np.nanpercentile(spy.k_change,q=10))


QQQ 2020-05-19 00:00:00 0.2091203796557024 -0.45913701443037197
NVO 2020-05-19 00:00:00 -0.09417990905274432 -0.47456756599687605
AZN 2020-05-19 00:00:00 -0.2415376833695585 -0.5116273204951323
MBUU 2020-05-19 00:00:00 0.27503665048482895 -0.4364955204247237
CWH 2020-05-19 00:00:00 0.11674096531962608 -0.5164024279176075


In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1

In [105]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

hi


In [148]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format(holdings[0])#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [152]:
pd.Series(response.json()[0]['date'])

0    2020-05-19
dtype: object

In [164]:
spy.iloc[-1]['Date']


Timestamp('2020-05-19 00:00:00')

In [93]:
np.nanpercentile(spy.k_change,q=10)

-0.45913701443037197